<a href="https://colab.research.google.com/github/zeekx/build_dl_framework/blob/master/steps/step09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np

In [36]:
class Variable:
    def __init__(self, data):
        if data is not None:
          if not isinstance(data, np.ndarray):
            raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None


    def set_creator(self, func):
        self.creator = func


# a -> f(a) -> b
# Variable: b.creator -> f
# b.creator -> f -> f.input -> a

    def backward(self):
        fs = [self.creator]
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        while fs:
            f = fs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                fs.append(x.creator)



In [37]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [38]:
class Function:
    def __call__(self, input):
        self.input = input
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


In [39]:

class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


In [40]:
def square(x):
  return Square()(x)

def exp(x):
  return Exp()(x)

Give a default grad value(array(1.0) for the last variable

In [43]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))


y.backward()

print("x.grad\t", x.grad)

x.grad	 3.297442541400256
